In [1]:
import json
import bz2
import sys
import os
import pandas as pd
import time

In [233]:
WIKI_DATA_FILTERED = 'filtered_politician.json.bz2'
QUOTEBANK_POLITICIANS_WORLD= 'quotebank_politicians_world.json.bz2'
QUOTEBANK_POLITICIANS_USA= 'quotebank_politicians_USA.json.bz2'
quotebank_dir ='../../data/'
print(type(os.listdir(quotebank_dir)))
quotebank_files=list( os.path.join(quotebank_dir,x) for x in os.listdir(quotebank_dir))
print(quotebank_files)

<class 'list'>
['../../data/quotes-2020.json.bz2']


In [234]:
qids_df=pd.read_json(WIKI_DATA_FILTERED, lines=True)
qids_df.head()

,qid,name,nationality
0,Q207,George W. Bush,Q30
1,Q946,Donald Tusk,Q36
2,Q1058,Narendra Modi,Q668
3,Q1253,Ban Ki-moon,Q884
4,Q3996,V. P. Kalairajan,Q668


In [235]:
qids_us_df = qids_df[qids_df['nationality']=='Q30']
qids_us_df.head()

,qid,name,nationality
0,Q207,George W. Bush,Q30
5,Q4496,Mitt Romney,Q30
7,Q11124,Stephen Breyer,Q30
9,Q11674,David Paterson,Q30
23,Q22201,Jim DeMint,Q30


In [236]:
qid_list = list(qids_df.qid)
qid_list_us = list(qids_us_df.qid)

In [237]:
CHUNKSIZE = 100000
try:
    os.remove(QUOTEBANK_POLITICIANS_WORLD)
    os.remove(QUOTEBANK_POLITICIANS_USA)
except:
    pass
total_time, sum_quotes_world, sum_quotes_usa, chunk_number=0,0,0,0

with bz2.open(QUOTEBANK_POLITICIANS_WORLD, 'wb') as bzout_world:
    with bz2.open(QUOTEBANK_POLITICIANS_USA, 'wb') as bzout_usa:
        for file in quotebank_files:
            with pd.read_json(file, lines=True, compression='bz2', chunksize=CHUNKSIZE) as df_reader:
                for chunk in df_reader:
                    t1=time.time()
                    chunk_not_empty = chunk['qids'].apply(lambda x: len(x)) != 0
                    chunk_politician = chunk[chunk_not_empty]['qids'].apply(lambda x: x[0]).isin(qid_list)
                    chunk_politician_us = chunk[chunk_not_empty]['qids'].apply(lambda x: x[0]).isin(qid_list_us)
                    quote_world = chunk[chunk_not_empty & chunk_politician]
                    quote_usa = chunk[chunk_not_empty & chunk_politician_us]
                    quote_world.to_json(path_or_buf=bzout_world,orient='records',lines=True)
                    quote_usa.to_json(path_or_buf=bzout_usa,orient='records',lines=True)
                    sum_quotes_world += len(quote_world)
                    sum_quotes_usa += len(quote_usa)
                    t2=time.time()
                    dt=t2-t1
                    total_time += dt
                    chunk_number += 1
                    print(chunk_number)
                    print("Dumped {} political quotations and {} US political quotations among {} quotations [quotations/s: {:.2f}]".format(len(quote_world), len(quote_usa), chunk_number*CHUNKSIZE, sum_quotes_world / dt))
                    

1
Dumped 15531 political quotations and 7678 US political quotations among 100000 quotations [quotations/s: 10019.58]
2
Dumped 15481 political quotations and 7693 US political quotations among 200000 quotations [quotations/s: 18201.97]
3
Dumped 15381 political quotations and 7628 US political quotations among 300000 quotations [quotations/s: 23765.77]
4
Dumped 15521 political quotations and 7545 US political quotations among 400000 quotations [quotations/s: 34074.78]
5
Dumped 15397 political quotations and 7465 US political quotations among 500000 quotations [quotations/s: 46660.20]
6
Dumped 15438 political quotations and 7691 US political quotations among 600000 quotations [quotations/s: 54923.40]
7
Dumped 15474 political quotations and 7669 US political quotations among 700000 quotations [quotations/s: 64093.94]
8
Dumped 15401 political quotations and 7726 US political quotations among 800000 quotations [quotations/s: 70015.10]
9
Dumped 15543 political quotations and 7589 US politica

In [238]:
print(f'WORLD: total quotes: {sum_quotes_world} in {total_time}s')
print(f'USA: total quotes: {sum_quotes_usa} in {total_time}s')

WORLD: total quotes: 812900 in 88.9895088672638s
USA: total quotes: 401991 in 88.9895088672638s


In [45]:
sumquote=0
with bz2.open(QUOTEBANK_POLITICIANS_USA, 'rt', encoding='utf8') as bzin_usa:
    for i, item in enumerate(bzin_usa):
        sumquote+=1
        if i%200 == 0: print(item)
    print(sumquote)

{"quoteID": "2015-08-31-000271", "quotation": "... a great day for veterans here in Littleton and all across the North Country. The men and women of our armed forces never fail our nation on the battlefield, and we must not fail them when they return home and enter civilian life.", "speaker": "Jeanne Shaheen", "qids": ["Q270316"], "date": "2015-08-31 02:10:00", "numOccurrences": 2, "probas": [["Jeanne Shaheen", "0.742"], ["None", "0.2359"], ["Kelly Ayotte", "0.0221"]], "urls": ["http://www.unionleader.com/article/20150831/NEWS12/150839902&source=RSS", "http://unionleader.com/article/20150831/NEWS12/150839902"], "phase": "E"}

{"quoteID": "2015-10-20-018726", "quotation": "confident that the courts ultimately will dismiss this taxpayer-funded political stunt, which would make healthcare more expensive for millions of Americans.", "speaker": "Katie Hill", "qids": ["Q58416634", "Q6377460"], "date": "2015-10-20 19:15:47", "numOccurrences": 2, "probas": [["Katie Hill", "0.9393"], ["None", "